# 1.Pré tratamento dos dados para análise.

In [17]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

In [18]:
df=pd.read_csv(r"Cuspidão\Completao.csv",delimiter=';',encoding="latin-1",decimal=',')

# Criar variáveis gerais

In [19]:
#Cria um variável com o nome de todos os setores do dataset e cria uma flag para se tem somente essa empresa no setor
ListofCompanies=df.drop_duplicates(subset=['SETOR_ATIV','Nome Empresa'])
ListofCompanies=ListofCompanies['SETOR_ATIV'].value_counts()
ListofCompanies=pd.DataFrame(ListofCompanies)
Listof1Companies=ListofCompanies[ListofCompanies['SETOR_ATIV']<4]
ListofCompanies=ListofCompanies[ListofCompanies['SETOR_ATIV']>=4]
ListofCompanies['Flag1Company']=0
Listof1Companies['Flag1Company']=1
ListofCompanies=pd.concat([ListofCompanies,Listof1Companies])
ListofCompanies['SETOR_ATIV']=ListofCompanies.index

#Cria Variáveis de Cuspir
DF_Polyline=pd.DataFrame()
DF_Cuspidao=pd.DataFrame()
DF_Cuspidao2=pd.DataFrame()
DF_Cuspidao_BAU=pd.DataFrame()
DF_Cuspidao_RESTO=pd.DataFrame()
df_produ_kmeans_REST_ALL=pd.DataFrame()

# Pré tratar dados para modelos


In [20]:
#Criar ano e mes
df['Ano']=df['DT_REFER'].apply(str).str[0:4]
df['Mes']=df['DT_REFER'].apply(str).str[5:7]
#Transformar colunas negativas em positivas
df['Passivo Circulante']=df['Passivo Circulante']*-1
df['Passivo Total']=df['Passivo Total']*-1
#Coloca a Flag de 1 company no dataset central
df1 = pd.merge( ListofCompanies,df, left_on='SETOR_ATIV', right_on='SETOR_ATIV')


In [21]:
df_produ=df1

In [22]:
#Transforma as relações de indicadores para clusterizar em numeros, para o for andar de lado.
df_produ[14]=df_produ['Ativo Circulante'].clip(lower=0)
df_produ[13]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[12]=df_produ['Passivo Circulante'].clip(upper=0)
df_produ[11]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[10]=df_produ['Passivo Total'].clip(upper=0)
df_produ[9]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[8]=df_produ['Ativo Total'].clip(lower=0)
df_produ[7]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[6]=df_produ['Reservas de Lucro'].clip(lower=0)
df_produ[5]=df_produ['Receita'].clip(lower=0)
df_produ[4]=df_produ['Receita'].clip(lower=0)
df_produ[3]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[2]=df_produ['Receita'].clip(lower=0)
df_produ[1]=df_produ['Lucro/Prejuízo do Período'].clip(lower=0)

# Clusterizar as empresas com dados realizados

In [23]:
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
#BAU KMEANS
for y in range(1,13):
    for x in ListofCompanies.index.values:
        if y==1:
            y1=1
            y2=2
        else:
            y1=y+1
            y2=y+2
        #filtra o setor
        df_produ_kmeans=df_provisorio[(df_provisorio['SETOR_ATIV']==x)&(df_provisorio['Flag1Company']==0)].reset_index(drop=True)
      
        Tcks_count=df_produ_kmeans.drop_duplicates(subset=['Ticker','DT_REFER'])
        Tcks_count=Tcks_count['DT_REFER'].value_counts()
        Tcks_count_mean=Tcks_count.mean()
        Tcks_count=pd.DataFrame(Tcks_count)
        ListOFTRI=Tcks_count[Tcks_count['DT_REFER']>=Tcks_count_mean]
        ListOFTRI['DATE']=ListOFTRI.index
        AnoMAX=ListOFTRI['DATE'].apply(str).str[0:4].max()
        MesMAX=ListOFTRI['DATE'].apply(str).str[5:7].max()

        #Filtrar ano e mês (ultimo tri de cada setor)
        df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans["Ano"]==AnoMAX)&(df_produ_kmeans["Mes"]==MesMAX)].reset_index(drop=True)      
        #Clusteriza as empresas com valores negativos como 0 (ruim)
        df_produ_kmeans_REST_NO=df_produ_kmeans[(df_produ_kmeans[y1]==0)|(df_produ_kmeans[y2]==0)].reset_index(drop=True)
        df_produ_kmeans_REST_NO[f"{y1}{y2} Cluster"]=0
        #Separa as empresas que são elegíveis a escalabilidade e a clusterização
        df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans[y1]!=0)&(df_produ_kmeans[y2]!=0)].reset_index(drop=True)
        df_produ_kmeans=df_produ_kmeans.sort_values(by=[y1,y2]).reset_index(drop=True)
        
        #se tiver mais de 1 amostra (empresa)
        if len(df_produ_kmeans)>=2:
            #FAÇO A ESCALABILIDADE COM STANDARD
            scaler = StandardScaler() 
            pca = PCA(n_components = 2) 
            features= df_produ_kmeans[[y1,y2]]
            scaled_df = scaler.fit_transform(features) 
            normalized_df = normalize(scaled_df) 
            normalized_df = pd.DataFrame(normalized_df)
            #E DEPOIS REDUZO AS DIMENSÕES E CORRELAÇÕES COM O PCA
            X_principal = pca.fit_transform(normalized_df) 
            features = pd.DataFrame(X_principal) 
            features.columns = ['P1', 'P2']
            c=3
            if len(features)<3:
                c=len(features)
                
            #DOU UM FIT NO KMEANS
            kmeans = KMeans(n_clusters=c, init='random',
                        n_init=10,
                        max_iter=300,random_state=0)
            kmeans.fit(features)
            #DPS EU FAÇO A CLUSTERIZAÇÃO
            d = KMeans(n_clusters=c, init='random',
                        n_init=10,
                        max_iter=300,random_state=0).fit_predict(features)
            df_produ_kmeans[f"{y1}{y2} Cluster"]=d
            DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
            DF_Cuspidao_BAU=pd.concat([DF_Cuspidao2,DF_Cuspidao_BAU]).reset_index(drop=True)
        else:
            DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
            DF_Cuspidao_BAU=pd.concat([DF_Cuspidao2,DF_Cuspidao_BAU]).reset_index(drop=True)
    df_provisorio=DF_Cuspidao_BAU
    DF_Cuspidao2=pd.DataFrame()
    DF_Cuspidao_BAU=pd.DataFrame()
    
DF_Cuspidao_BAU=df_provisorio
DF_Cuspidao=pd.DataFrame()
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
#REJEITADOS KMEANS
for y in range(1,13):
    if y==1:
        y1=1
        y2=2
    else:
        y1=y+1
        y2=y+2
    #filtra o setor
    df_produ_kmeans=df_provisorio[(df_provisorio['Flag1Company']==1)].reset_index(drop=True)
    Tcks_count=df_produ_kmeans.drop_duplicates(subset=['Ticker','DT_REFER'])
    Tcks_count=Tcks_count['DT_REFER'].value_counts()
    Tcks_count_mean=Tcks_count.mean()
    Tcks_count=pd.DataFrame(Tcks_count)
    ListOFTRI=Tcks_count[Tcks_count['DT_REFER']>=Tcks_count_mean]
    ListOFTRI['DATE']=ListOFTRI.index
    AnoMAX=ListOFTRI['DATE'].apply(str).str[0:4].max()
    MesMAX=ListOFTRI['DATE'].apply(str).str[5:7].max()
    #Filtrar ano e mês (ultimo tri de cada setor)
    df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans["Ano"]==AnoMAX)&(df_produ_kmeans["Mes"]==MesMAX)].reset_index(drop=True)
    #Clusteriza as empresas com valores negativos como 0 (ruim)
    df_produ_kmeans_REST_NO=df_produ_kmeans[(df_produ_kmeans[y1]==0)|(df_produ_kmeans[y2]==0)].reset_index(drop=True)
    df_produ_kmeans_REST_NO[f"{y1}{y2} Cluster"]=0
    #Separa as empresas que são elegíveis a escalabilidade e a clusterização
    df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans[y1]!=0)&(df_produ_kmeans[y2]!=0)].reset_index(drop=True)
    df_produ_kmeans=df_produ_kmeans.sort_values(by=[y1,y2]).reset_index(drop=True)
    #se tiver mais de 1 amostra (empresa)
    if len(df_produ_kmeans)>=2:
        #FAÇO A ESCALABILIDADE COM STANDARD
        scaler = StandardScaler() 
        pca = PCA(n_components = 2) 
        features= df_produ_kmeans[[y1,y2]]
        scaled_df = scaler.fit_transform(features) 
        normalized_df = normalize(scaled_df) 
        normalized_df = pd.DataFrame(normalized_df)
        #E DEPOIS REDUZO AS DIMENSÕES E CORRELAÇÕES COM O PCA
        X_principal = pca.fit_transform(normalized_df) 
        features = pd.DataFrame(X_principal) 
        features.columns = ['P1', 'P2']
        c=3
        if len(features)<3:
            c=len(features)
        #DOU UM FIT NO KMEANS
        kmeans = KMeans(n_clusters=c, init='random',
                    n_init=10,
                    max_iter=300,random_state=0)
        kmeans.fit(features)
        #DPS EU FAÇO A CLUSTERIZAÇÃO
        d = KMeans(n_clusters=c, init='random',
                    n_init=10,
                    max_iter=300,random_state=0).fit_predict(features)
        df_produ_kmeans[f"{y1}{y2} Cluster"]=d
        DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
        DF_Cuspidao_RESTO=pd.concat([DF_Cuspidao2,DF_Cuspidao_RESTO]).reset_index(drop=True)
    else:
        DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
        DF_Cuspidao_RESTO=pd.concat([DF_Cuspidao2,DF_Cuspidao_RESTO]).reset_index(drop=True)
    df_provisorio=DF_Cuspidao_RESTO
    DF_Cuspidao2=pd.DataFrame()
    DF_Cuspidao_RESTO=pd.DataFrame()
DF_kmeans=pd.concat([DF_Cuspidao_BAU,df_provisorio]).reset_index(drop=True)
DF_kmeans["Dados"]="Realizado"

In [24]:
DF_kmeans=DF_kmeans[['Nome Empresa','Ticker','SETOR_ATIV','DT_REFER',"Dados",
             1,2,3,4,5,6,7,8,9,10,11,12,13,14,
            '12 Cluster',
            '34 Cluster',
            '56 Cluster',
            '78 Cluster',
            '910 Cluster',
            '1112 Cluster',
            '1314 Cluster',
             ]]

# Prever os próximos 4 tri

In [25]:
#Separo os indicadores que eu quero prever
df_produ[11]=df_produ['Obrigações Fiscais']
df_produ[10]=df_produ['Obrigações Sociais e Trabalhistas']
df_produ[9]=df_produ['Contas a Receber']
df_produ[8]=df_produ['Ativo Circulante']
df_produ[7]=df_produ['Passivo Circulante']
df_produ[6]=df_produ['Passivo Total']
df_produ[5]=df_produ['Ativo Total']
df_produ[4]=df_produ['Reservas de Lucro']
df_produ[3]=df_produ['Patrimônio Líquido']
df_produ[2]=df_produ['Receita']
df_produ[1]=df_produ['Lucro/Prejuízo do Período']

In [26]:
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11]]

#pego a lista de empresas para analisar
LitsofTickers=df_provisorio.drop_duplicates(subset='Ticker')
LitsofTickers=LitsofTickers['Ticker'].values
for x in LitsofTickers:
    DF_Poly=df_provisorio[df_provisorio['Ticker']==x].reset_index(drop=True)
    DF_Poly.sort_values(by=['DT_REFER']).reset_index(drop=True)
    #transformo os indexes em variáveis e a data em datetime
    DF_Poly["indexes"]=DF_Poly.index
    DF_Poly['DT_REFER']=pd.to_datetime(DF_Poly['DT_REFER'])
    #pego a ultima linha dessa empresa para pegar as informações principais para criar um novo dataset
    Lastmonth=DF_Poly.iloc[-1]['DT_REFER']
    Lastname=DF_Poly.iloc[-1]['Nome Empresa']
    Lastsector=DF_Poly.iloc[-1]['SETOR_ATIV']
    Lastnamesub=DF_Poly.iloc[-1]['Ticker']
    Lastindex=DF_Poly.iloc[-1]['indexes']
    #Crio as datas que eu quero ver la na frente,baseado na data do ultimo balanço
    Lastmonth1=Lastmonth+ relativedelta(months=+3)
    Lastmonth2=Lastmonth+ relativedelta(months=+6)
    Lastmonth3=Lastmonth+ relativedelta(months=+9)
    Lastmonth4=Lastmonth+ relativedelta(months=+12)
    #Crio um dataframe novo com essas datas
    Dateto=pd.DataFrame({'Data_REF': [Lastmonth1,Lastmonth2,Lastmonth3,Lastmonth4]})
    X_seq=pd.DataFrame({'x': [Lastindex+1,Lastindex+2,Lastindex+3,Lastindex+4]})
    #Configuro o modelo polimonial
    polyreg=make_pipeline(PolynomialFeatures(3),LinearRegression())
    Pred=Dateto
    Pred["Nome Empresa"]=Lastname
    Pred['Ticker']=Lastnamesub
    Pred['SETOR_ATIV']=Lastsector
    #Leio o historico de todos os indicadores e faço a previsão
    for y in range(1,12):
        yy=DF_Poly[['indexes']]
        xx=DF_Poly[[y]]
        polyreg.fit(yy,xx)
        Pred2=polyreg.predict(X_seq)
        Pred2=pd.DataFrame(Pred2,columns=[y])
        Pred[y]=Pred2
    DF_Polyline=pd.concat([Pred,DF_Polyline]).reset_index(drop=True)

    
DF_Polyline['Ano']=DF_Polyline['Data_REF'].apply(str).str[0:4]
DF_Polyline['Mes']=DF_Polyline['Data_REF'].apply(str).str[5:7]
DF_Polyline['Dados']="Previsão"

# Previsão de dividendos + cuspir json

In [269]:
dividendos=pd.read_csv(fr"Cuspidão\cuspidor_div.csv",decimal=',',sep=';', encoding='latin-1')

In [296]:
tick=dividendos['tick'].drop_duplicates()

for name in tick:
    #faço os tratame
    DF_Polyline=pd.DataFrame()
    DF_Poly=dividendos[dividendos['tick']==name].reset_index(drop=True)

    DF_Poly['Valor pago']=DF_Poly['Valor pago']/DF_Poly['Qauntidade de Ações']
    DF_Poly['Data']=pd.to_datetime(DF_Poly['Data'])
    DF_Poly=DF_Poly.drop(columns=['Data de pagamento','Qauntidade de Ações'])

    #agrupo as informações em trimestres
    DF_Poly_trimestre = DF_Poly.resample('Q', on='Data').sum()

    DF_Poly_trimestre['data']=DF_Poly_trimestre.index
    DF_Poly_trimestre=DF_Poly_trimestre.reset_index(drop=True)
    DF_Poly_trimestre=DF_Poly_trimestre.sort_values(by=['data']).reset_index(drop=True)
    DF_Poly_trimestre["indexes"]=DF_Poly_trimestre.index
    DF_Poly_trimestre['tipo']=DF_Poly['tick'].str.slice(4, 5)[0]
    Lastmonth=DF_Poly_trimestre.iloc[-1]['data']
    Lasttype=DF_Poly_trimestre.iloc[-1]['tipo']
    Lastindex=DF_Poly_trimestre.iloc[-1]['indexes']

    Lastmonth1=Lastmonth+ relativedelta(months=+3)
    Lastmonth2=Lastmonth+ relativedelta(months=+6)
    Lastmonth3=Lastmonth+ relativedelta(months=+9)
    Lastmonth4=Lastmonth+ relativedelta(months=+12)
    Lastmonth5=Lastmonth+ relativedelta(months=+15)
    Lastmonth6=Lastmonth+ relativedelta(months=+18)
    Lastmonth7=Lastmonth+ relativedelta(months=+21)

     #Crio um dataframe novo com essas datas
    Dateto=pd.DataFrame({'data': [Lastmonth1,Lastmonth2,Lastmonth3,Lastmonth4,Lastmonth5,Lastmonth6,Lastmonth7]})
    X_seq=pd.DataFrame({'x': [Lastindex+1,Lastindex+2,Lastindex+3,Lastindex+4,Lastindex+5,Lastindex+6,Lastindex+7]})
    #Configuro o modelo polimonial
    polyreg=make_pipeline(PolynomialFeatures(3),LinearRegression())
    Pred=Dateto
    Pred['tipo']=Lasttype
    #Leio o historico de todos os indicadores e faço a previsão

    yy=DF_Poly_trimestre[['indexes']]
    xx=DF_Poly_trimestre['Valor pago']
    polyreg.fit(yy,xx)
    Pred2=polyreg.predict(X_seq)
    Pred2=pd.DataFrame(Pred2,columns=['Valor pago'])
    Pred['Valor pago']=Pred2

    DF_Polyline=pd.concat([Pred,DF_Polyline]).reset_index(drop=True)
    DF_Polyline['info']="prev"

    DF_Poly_trimestre=DF_Poly_trimestre.drop(columns=['indexes'])
    DF_Poly_trimestre["info"]="real"
    DF_Poly_trimestre[['data','tipo','Valor pago','info']]
    DF_Poly_trimestre=DF_Poly_trimestre.append(DF_Polyline).reset_index(drop=True)
    DF_Poly_trimestre=DF_Poly_trimestre.sort_values(by=['data']).reset_index(drop=True)
    DF_Poly_trimestre["index"]=DF_Poly_trimestre.index
    DF_Poly_trimestre['period']='tri'
    DF_Poly_trimestre=DF_Poly_trimestre.rename(columns={"Valor pago": "valor"})
    DF_Poly_trimestre[['tipo','info','period','valor','data','index']]
    DF_Poly_trimestre['data']=pd.to_datetime(DF_Poly_trimestre['data']).dt.date
    DF_Poly_trimestre['data']=DF_Poly_trimestre['data'].apply(str)
    dados = DF_Poly_trimestre.to_dict('records')
    with open(f'pipeline/Empresas_data/{name[:4]}_dividends.json', 'w', encoding='utf-8') as arquivo:
            json.dump(dados, arquivo, ensure_ascii=False)

# Organizar os dados para cuspir no Front

In [27]:
DF_kmeans=DF_kmeans.fillna(0)
#Separo as colunas principais do dataset Kmeans e dou um replace nas clusterizações para criar uma nota
DF_kmeans_2=DF_kmeans.loc[:,('Ticker','DT_REFER',"Dados",'12 Cluster',
            '34 Cluster',
            '56 Cluster',
            '78 Cluster',
            '910 Cluster',
            '1112 Cluster',
            '1314 Cluster')].replace([0,1,2],[0,10,5])
#Crio a nota da empresa baseada nos clusters (0 é ruim, 1 é bom e 2 é mediano)
DF_kmeans_2['Nota da Empresa']=DF_kmeans_2['12 Cluster']+DF_kmeans_2['34 Cluster']+DF_kmeans_2['56 Cluster']+DF_kmeans_2['78 Cluster']+DF_kmeans_2['910 Cluster']+DF_kmeans_2['1112 Cluster']+DF_kmeans_2['1314 Cluster']
DF_kmeans_2['Nota da Empresa']=DF_kmeans_2['Nota da Empresa']/7
DF_kmeans_2['Desempenho do Trimestre']=np.where(DF_kmeans_2['Nota da Empresa']<5,"Ruim",
                                        (np.where((DF_kmeans_2['Nota da Empresa']>=5) & (DF_kmeans_2['Nota da Empresa']<7) ,"Médio","Bom")))
#Pego o indicador com a melhor nota de clusterização (indicador denominador)
DF_kmeans_2['Melhor Indicador1']=(DF_kmeans_2['12 Cluster']+DF_kmeans_2['34 Cluster']+DF_kmeans_2['56 Cluster'])/3
DF_kmeans_2['Melhor Indicador2']=(DF_kmeans_2['34 Cluster']+DF_kmeans_2['78 Cluster']+DF_kmeans_2['910 Cluster']+DF_kmeans_2['1112 Cluster']+DF_kmeans_2['1314 Cluster'])/5
DF_kmeans_2['Melhor Indicador']=np.where(DF_kmeans_2['Melhor Indicador1']>DF_kmeans_2['Melhor Indicador2'],"Receita",
                                        (np.where(DF_kmeans_2['Melhor Indicador1']<DF_kmeans_2['Melhor Indicador2'],"Patrimônio Líquido","Nenhum")))
df_produ=df1
#Pego as colunas antigas sem tratamento de clips
DF_kmeans_2 = pd.merge(df_produ,DF_kmeans_2 ,left_on=['Ticker','DT_REFER'], right_on=['Ticker','DT_REFER'])
DF_kmeans_2['Analise']=DF_kmeans_2['Flag1Company'].replace([0,1],["Analise por Setor",'Analise com outras empresas'])
DF_kmeans_2['Tipo de Analise']='Kmeans'
#Deixo tudo limpinho e arrumado no dataset kmeans
DF_kmeans_2=DF_kmeans_2[['Nome Empresa','Ticker','SETOR_ATIV','Dados','Analise','Tipo de Analise','Melhor Indicador','Desempenho do Trimestre','DT_REFER','Lucro/Prejuízo do Período','Ativo Total','Ativo Circulante','Contas a Receber','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas','Obrigações Fiscais','Reservas de Lucro','Patrimônio Líquido','Receita']]
#Dou nome aos bois no dataset do polimonial
DF_Polyline_2=DF_Polyline.rename({'Data_REF': 'DT_REFER', 
                                  1: 'Lucro/Prejuízo do Período',
                                  2: 'Receita', 
                                  3: 'Patrimônio Líquido', 
                                  4: 'Reservas de Lucro', 
                                  5: 'Ativo Total',
                                  6: 'Passivo Total', 
                                  7: 'Passivo Circulante', 
                                  8: 'Ativo Circulante',
                                  9: 'Contas a Receber',
                                  10: 'Obrigações Sociais e Trabalhistas',
                                  11: 'Obrigações Fiscais',
                                 }, axis=1)
#Seto as mesmas colunas do kmeans
DF_Polyline_2['Melhor Indicador']='Não Analisado'
DF_Polyline_2['Desempenho do Trimestre']='Não Analisado'
DF_Polyline_2['Nota da Empresa']='Não Analisado'
DF_Polyline_2['Analise']='Analise de Historico Financeiro'
DF_Polyline_2['Tipo de Analise']='Regressão Polimonial'
#Deixo tudo limpinho e arrumado no dataset polimonial
DF_Polyline_2=DF_Polyline_2[['Nome Empresa','Ticker','SETOR_ATIV','Dados','Analise','Tipo de Analise','Melhor Indicador','Desempenho do Trimestre','DT_REFER','Lucro/Prejuízo do Período','Ativo Total','Ativo Circulante','Contas a Receber','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas','Obrigações Fiscais','Reservas de Lucro','Patrimônio Líquido','Receita']]
#Deipois junto tudo e cuspo
Base_Unica=pd.concat([DF_Polyline_2,DF_kmeans_2]).reset_index(drop=True)
Base_Unica.to_csv(rf"Cuspidão\Base_Unica.csv",decimal=',',sep=';', encoding='latin-1',index=False)

# Código do vitor

# Cruzar bases e ter uma base unica

# PIPELINE CUSPIDOR

# Ajustando setor

In [274]:
import re
import json
from datetime import datetime


In [275]:
setor_agrup=pd.read_csv(fr"Depara\setor_ajust.csv",delimiter=';',encoding="utf-8",decimal='.')


In [276]:
#Precisa multiplicar os valores da empresa
#Base_Unica['Patrimônio Líquido']=Base_Unica['Patrimônio Líquido']*1000

In [277]:
Base_Unica_json = pd.merge( Base_Unica, setor_agrup, left_on='SETOR_ATIV', right_on='SETOR_ASIS')
Base_Unica_json = Base_Unica_json.drop(columns=['SETOR_ATIV','SETOR_ASIS'])

# criar arquivo unico de leitura dos cards

In [278]:
Base_Unica_CARDS_JSON=Base_Unica_json[Base_Unica_json['Dados']=='Realizado'].reset_index(drop=True)
Base_Unica_CARDS_JSON = Base_Unica_CARDS_JSON.groupby(['Ticker','Nome Empresa','Patrimônio Líquido','SETOR_AJUST','Desempenho do Trimestre'])['DT_REFER'].max().reset_index()

In [279]:
#Cria a coluna PL do card
def formatar_valor(valor):
    if valor >= 1000000000:  # Valor em bilhões
        valor_dividido = valor / 1000000000
        return f'{round(valor_dividido,2):.2f} BI'
    
    elif valor >= 1000000:  # Valor em milhões
        valor_dividido = valor / 1000000
        return f'{round(valor_dividido,2):.2f} MM'
    
    elif valor >= 1000: # Valor menor que 1 milhão
        valor_dividido = valor / 1000
        return f'{round(valor_dividido,2):.2f} Mil'
    
    elif valor < 1000 and valor > -1000000: # Valor menor que 1 milhão
        valor_dividido = valor / 1000 
        return f'{round(valor_dividido,2):.2f} Mil'
    
    if valor <= -1000000000:  # Valor em bilhões
        valor_dividido = valor / 1000000000
        return f'{round(valor_dividido,2):.2f} BI'
    
    elif valor <= -1000000:  # Valor em milhões
        valor_dividido = valor / 1000000
        return f'{round(valor_dividido,2):.2f} MM'
    
    


In [280]:
Base_Unica_CARDS_JSON['PL'] = Base_Unica_CARDS_JSON ['Patrimônio Líquido'].apply(formatar_valor)

In [281]:
#cria a coluna symbol, PM, gain e CMP
#coluna PM depende do Preço médio
#coluna symbol:down,side e up
#coluna CMP: YoY e MoM

Base_Unica_CARDS_JSON['symbol'] = "side"
Base_Unica_CARDS_JSON['PM'] = "0.0"
Base_Unica_CARDS_JSON['gain'] = "0%"
Base_Unica_CARDS_JSON['CMP'] = "YoY"


In [282]:
#cria a coluna color

def map_desempenho(valor):
    if valor == 'Bom':
        return 'green'
    elif valor == 'Médio':
        return 'orange'
    elif valor == 'Ruim':
        return 'red'
    else:
        return valor

Base_Unica_CARDS_JSON['Desempenho do Trimestre'] = Base_Unica_CARDS_JSON['Desempenho do Trimestre'].apply(map_desempenho)

In [283]:
#faz os ajustes finais
Base_Unica_CARDS_JSON = Base_Unica_CARDS_JSON.rename(columns={'Ticker':'tick',
                                                              'Nome Empresa':'nome',
                                                             'SETOR_AJUST':'sector',
                                                             'Desempenho do Trimestre':'color'
                                                             })
Base_Unica_CARDS_JSON=Base_Unica_CARDS_JSON.drop(columns=['Patrimônio Líquido','DT_REFER'])


In [284]:


# Extrai a parte específica da string usando uma expressão regular
Base_Unica_CARDS_JSON['nome'] = Base_Unica_CARDS_JSON['nome'].str.replace(' - EM RECUPERAÇÃO JUDICIAL', '')
Base_Unica_CARDS_JSON['nome'] = Base_Unica_CARDS_JSON['nome'].str.replace('S.A.', '')
Base_Unica_CARDS_JSON['nome'] = Base_Unica_CARDS_JSON['nome'].str.replace('SA.', '')
Base_Unica_CARDS_JSON['nome'] = Base_Unica_CARDS_JSON['nome'].str.replace('S/A.', '')
Base_Unica_CARDS_JSON['nome'] = Base_Unica_CARDS_JSON['nome'].str.title()

# Salva os Jsons


In [285]:

# Converta o DataFrame selecionado em uma lista de dicionários
dados = Base_Unica_CARDS_JSON.to_dict('records')

# Salve os dados como um arquivo JSON

with open('pipeline/cards_main.json', 'w', encoding='utf-8') as arquivo:
    json.dump(dados, arquivo, ensure_ascii=False)

# Pipeline tela unica empresa

### cuspir informações de finanças

In [286]:
inf_company=pd.read_csv(fr"Cuspidão\cuspidor_inf.csv",delimiter=';',encoding="latin-1",decimal=',')

In [287]:
inf_company['tick_resume']=inf_company['tick'].str.slice(0, 4)

In [288]:
tickador=inf_company['tick_resume'].drop_duplicates()



for x in tickador:
    data=inf_company[inf_company['tick_resume']==x]
    
    #Pego o PL médio
    PL=data[data['Info']=='P/L']
    PL=PL['Data'].str.replace('.', '').str.replace(',', '.')
    PL=pd.to_numeric(PL).mean()
    
    #Pego o DivYield
    divyield=data[data['Info']=='Div. Yield']
    divyield=divyield['Data'].str.replace('.', '').str.replace(',', '.').str.replace('%', '')
    divyield=pd.to_numeric(divyield).mean()
    
    #Pego o ROE
    ROE=data[data['Info']=='ROE']
    ROE=ROE['Data'].str.replace('.', '').str.replace(',', '.').str.replace('%', '')
    ROE=pd.to_numeric(ROE).mean()
    
    #Pego a cresc.receita
    CR5=data[data['Info']=='Cres. Rec (5a)']
    CR5=CR5['Data'].str.replace('.', '').str.replace(',', '.').str.replace('%', '')
    CR5=pd.to_numeric(CR5).mean()
    
    #Pego a LPA
    LPA=data[data['Info']=='LPA']
    LPA=LPA['Data'].str.replace('.', '').str.replace(',', '.')
    LPA=pd.to_numeric(LPA).mean()
    
    #Pego a Lucro.Liq
    LucroLiqu=data[data['Info']=='Relatorio 3 meses Lucro Líquido']
    LucroLiqu=LucroLiqu['Data'].str.replace('.', '').str.replace(',', '.')
    LucroLiqu=pd.to_numeric(LucroLiqu).mean()
    LucroLiqu=formatar_valor(LucroLiqu)
    
    #Pego a Lucro.Liq
    Valormercado=data[data['Info']=='Valor de mercado']
    Valormercado=Valormercado['Data'].str.replace('.', '').str.replace(',', '.')
    Valormercado=pd.to_numeric(Valormercado).mean()
    Valormercado=formatar_valor(Valormercado)
    
    
    #Pego a data ulti.balan
    UBL=data[data['Info']=='Últ balanço processado']
    UBL=UBL['Data']
    # Faz o parsing da data no formato "01/01/2023"
    UBL= datetime.strptime(UBL.iloc[0], '%d/%m/%Y')   
    # Formata a data no formato "Jan-23"
    UBL = UBL.strftime('%b-%y')

    #Pego a MENOR COTAÇÃO
    MIN=data[data['Info']=='Min 52 sem']
    MIN=MIN['Data'].str.replace('.', '').str.replace(',', '.')
    MIN=pd.to_numeric(MIN).mean()
    
    #Pego a MAIOR COTAÇÃO
    MAX=data[data['Info']=='Max 52 sem']
    MAX=MAX['Data'].str.replace('.', '').str.replace(',', '.')
    MAX=pd.to_numeric(MAX).mean()

    new_row = pd.DataFrame({"tick": x, 
                            "DY":f"{round(divyield,2)}%", 
                            "PL":f"{round(PL,2)}",
                            "ROE":f"{round(ROE, 2)}%",
                            "CR5":f"{round(CR5,2)}%",
                            "LPA":f"{round(LPA,2)}",
                            "LucroLiqu":LucroLiqu,
                            "Valormercado":Valormercado,
                            "UBL":UBL,
                            "MIN":f"{round(MIN,2)}",
                            "MAX":f"{round(MAX,2)}"}, index=[0])
    # Converta o DataFrame selecionado em uma lista de dicionários
    dados = new_row.to_dict('records')

    # Salve os dados como um arquivo JSON

    with open(f'pipeline/Empresas_data/{x}_fundamentalist.json', 'w', encoding='utf-8') as arquivo:
        json.dump(dados, arquivo, ensure_ascii=False)

### cuspir informações de finanças para o chart

# Informações previsão 

In [289]:
Base_Unica_financas_JSON=Base_Unica_json[Base_Unica_json['Tipo de Analise']=='Regressão Polimonial'].reset_index(drop=True)
Base_Unica_financas_JSON_PL_prev = Base_Unica_financas_JSON.groupby(['Ticker','Patrimônio Líquido',
'Lucro/Prejuízo do Período',
'Receita',
'Passivo Total',
'Contas a Receber',
'Obrigações Sociais e Trabalhistas',
'Reservas de Lucro'])['DT_REFER'].max().reset_index()
Base_Unica_financas_JSON_PL_prev['info']='prev'
Base_Unica_financas_JSON_PL_prev['period']='tri'



# Informações realizado

In [290]:
df=pd.read_csv(r"Cuspidão\Completao.csv",delimiter=';',encoding="latin-1",decimal=',')

In [291]:
Base_Unica_financas_JSON_PL_real = df.groupby(['Ticker','Patrimônio Líquido',
'Lucro/Prejuízo do Período',
'Receita',
'Passivo Total',
'Contas a Receber',
'Obrigações Sociais e Trabalhistas',
'Reservas de Lucro'])['DT_REFER'].max().reset_index()
Base_Unica_financas_JSON_PL_real['info']='real'
Base_Unica_financas_JSON_PL_real['period']='tri'

In [292]:
base_company_indicators=Base_Unica_financas_JSON_PL_prev.append(Base_Unica_financas_JSON_PL_real).reset_index(drop=True)

In [293]:
indicators=['Patrimônio Líquido',
'Lucro/Prejuízo do Período',
'Receita',
'Passivo Total',
'Contas a Receber',
'Obrigações Sociais e Trabalhistas',
'Reservas de Lucro']
tick=Base_Unica_financas_JSON_PL_prev['Ticker'].drop_duplicates()

# Cuspidor de indicadores - dividendos

In [294]:
for x in indicators:
    for y in tick:
        #Tratar as informações:
        base_company_indicators_cuspidor=base_company_indicators[base_company_indicators['Ticker']==y].reset_index(drop=True)
        base_company_indicators_cuspidor['valor']=base_company_indicators_cuspidor[x]
        base_company_indicators_cuspidor = base_company_indicators_cuspidor.groupby(['Ticker','info','period','valor'])['DT_REFER'].max().reset_index()
        base_company_indicators_cuspidor['DT_REFER']=pd.to_datetime(base_company_indicators_cuspidor['DT_REFER']).dt.date
        base_company_indicators_cuspidor=base_company_indicators_cuspidor.sort_values(by=['DT_REFER'], ascending=True).reset_index(drop=True)
        base_company_indicators_cuspidor['index']=base_company_indicators_cuspidor.index
        base_company_indicators_cuspidor=base_company_indicators_cuspidor.rename(columns={"Ticker": "tick", 
                                                                                          "DT_REFER": "data"})
        
        base_company_indicators_cuspidor['data']=base_company_indicators_cuspidor['data'].apply(str)
        base_company_indicators_cuspidor['valor']=round(base_company_indicators_cuspidor['valor'],2)
        #Cuspir json das empresas
        dados = base_company_indicators_cuspidor.to_dict('records')
        name_file=x.replace('/','_')
        name_file=name_file.replace(' ','_')
        # Salve os dados como um arquivo JSON
        with open(f'pipeline/Empresas_data/{y}_{name_file}.json', 'w', encoding='utf-8') as arquivo:
            json.dump(dados, arquivo, ensure_ascii=False)

In [295]:
print(base_company_indicators_cuspidor)

    tick  info period       valor        data  index
0   YDUQ  real    tri   118285.00  2011-03-31      0
1   YDUQ  real    tri    99128.00  2011-06-30      1
2   YDUQ  real    tri    96016.00  2011-09-30      2
3   YDUQ  real    tri   147403.00  2012-06-30      3
4   YDUQ  real    tri   142601.00  2012-09-30      4
5   YDUQ  real    tri   226237.00  2013-09-30      5
6   YDUQ  real    tri   412826.00  2014-06-30      6
7   YDUQ  real    tri   412827.00  2014-09-30      7
8   YDUQ  real    tri   619056.00  2015-03-31      8
9   YDUQ  real    tri   622840.00  2015-09-30      9
10  YDUQ  real    tri   972570.00  2016-03-31     10
11  YDUQ  real    tri   955336.00  2016-06-30     11
12  YDUQ  real    tri   808906.00  2016-09-30     12
13  YDUQ  real    tri   669584.00  2017-03-31     13
14  YDUQ  real    tri   682036.00  2017-06-30     14
15  YDUQ  real    tri   682207.00  2017-09-30     15
16  YDUQ  real    tri   958058.00  2018-03-31     16
17  YDUQ  real    tri   886873.00  2018-06-30 